In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("player_with_game_id.csv")
gamesDf = pd.read_csv("Kaggle Data/game.csv")

In [3]:
gamesDf.keys()

Index(['season_id', 'team_id_home', 'team_abbreviation_home', 'team_name_home',
       'game_id', 'game_date', 'matchup_home', 'wl_home', 'min', 'fgm_home',
       'fga_home', 'fg_pct_home', 'fg3m_home', 'fg3a_home', 'fg3_pct_home',
       'ftm_home', 'fta_home', 'ft_pct_home', 'oreb_home', 'dreb_home',
       'reb_home', 'ast_home', 'stl_home', 'blk_home', 'tov_home', 'pf_home',
       'pts_home', 'plus_minus_home', 'video_available_home', 'team_id_away',
       'team_abbreviation_away', 'team_name_away', 'matchup_away', 'wl_away',
       'fgm_away', 'fga_away', 'fg_pct_away', 'fg3m_away', 'fg3a_away',
       'fg3_pct_away', 'ftm_away', 'fta_away', 'ft_pct_away', 'oreb_away',
       'dreb_away', 'reb_away', 'ast_away', 'stl_away', 'blk_away', 'tov_away',
       'pf_away', 'pts_away', 'plus_minus_away', 'video_available_away',
       'season_type'],
      dtype='object')

In [4]:
d = gamesDf[["game_id","game_date"]]
d[d["game_date"] == d["game_date"].min()]

,game_id,game_date
0,24600001,1946-11-01 00:00:00


In [5]:
# Convert to date time game_date column
df2 = gamesDf[["game_id","game_date"]]
df2['game_date'] = pd.to_datetime(df2['game_date'])
dfWithDate = df.merge(df2, on="game_id").reset_index()

/tmp/ipykernel_29695/3219061645.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['game_date'] = pd.to_datetime(df2['game_date'])


In [6]:
# The min date for which we have data
dfWithDate["game_date"].min()

Timestamp('1996-11-01 00:00:00')

In [7]:
# Group games and find all the players for a team in that game
grouped = df.groupby([dfWithDate['game_date'].dt.date, 'team_id_x','game_id'])['ilkid'].apply(list).reset_index()

In [8]:
# View of the table
grouped

,game_date,team_id_x,game_id,ilkid
0,1996-11-01,ATL,29600005,"[LAETTCH01, BLAYLMO01, BARRYJO01, NORMAKE01, C..."
1,1996-11-01,BOS,29600001,"[FOXRI01, WESLEDA01, ELLISPE01, WILLIER01, BRI..."
2,1996-11-01,CHI,29600001,"[BROWNRA01, JORDAMI01, RODMADE01, BUECHJU01, S..."
3,1996-11-01,CLE,29600002,"[GEARYRE01, THOMACA01, POTAPVI01, LANGAN02, HI..."
4,1996-11-01,DAL,29600010,"[HARPEDE01, DUMASTO01, GATLICH01, KIDDJA01, MI..."
...,...,...,...,...
45259,2021-11-06,LAL,42200235,[JAMESLE01]
45260,2021-11-06,LAL,42200236,[JAMESLE01]
45261,2021-11-08,LAL,42200312,[JAMESLE01]
45262,2021-11-08,MIA,42200403,[HASLEUD01]


In [9]:
playerRegularSeason = pd.read_csv("Data/player_regular_season.txt")
playerRegularSeason.keys()

Index(['ilkid', 'year', 'firstname', 'lastname', 'team', 'leag', 'gp',
       'minutes', 'pts', 'oreb', 'dreb', 'reb', 'asts', 'stl', 'blk',
       'turnover', 'pf', 'fga', 'fgm', 'fta', 'ftm', 'tpa', 'tpm'],
      dtype='object')

In [10]:
# Normalize the player stats by dividing stats by games played
playerRegularSeason1 = playerRegularSeason.iloc[:,:7]
playerRegularSeason2 = playerRegularSeason.iloc[:,7:]
playerRegularSeason2Nomarlized = playerRegularSeason2.div(playerRegularSeason1["gp"], axis=0).round(2)
playerRegularSeason2Nomarlized


,minutes,pts,oreb,dreb,reb,asts,stl,blk,turnover,pf,fga,fgm,fta,ftm,tpa,tpm
0,0.00,11.21,0.00,0.00,0.00,0.74,0.00,0.00,0.00,3.43,17.74,4.30,3.79,2.62,0.00,0.0
1,0.00,2.17,0.00,0.00,0.00,0.67,0.00,0.00,0.00,1.53,3.03,0.77,1.17,0.63,0.00,0.0
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.0
3,0.00,2.38,0.00,0.00,0.00,0.28,0.00,0.00,0.00,1.69,4.53,0.91,1.19,0.55,0.00,0.0
4,0.00,5.09,0.00,0.00,0.00,0.93,0.00,0.00,0.00,2.83,7.55,2.14,1.36,0.81,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19107,8.00,1.74,0.79,0.79,1.58,0.42,0.21,0.32,0.26,1.58,2.16,0.63,0.58,0.47,0.05,0.0
19108,15.82,3.91,1.80,3.09,4.89,0.38,0.18,0.87,0.82,1.73,3.64,1.58,1.31,0.76,0.02,0.0
19109,13.33,3.33,0.67,1.33,2.00,0.00,1.33,0.00,0.67,2.00,4.00,1.67,0.00,0.00,0.00,0.0
19110,9.00,2.33,0.00,0.33,0.33,1.00,1.33,0.00,1.00,0.67,3.67,1.00,0.33,0.33,1.33,0.0


In [11]:
# Rejoin and form the full table
playerRegularSeasonNormalized = pd.concat([playerRegularSeason1, playerRegularSeason2Nomarlized], axis=1)
playerRegularSeasonNormalized


,ilkid,year,firstname,lastname,team,leag,gp,minutes,pts,oreb,...,stl,blk,turnover,pf,fga,fgm,fta,ftm,tpa,tpm
0,ABRAMJO01,1946,John,Abramovic,PIT,N,47,0.00,11.21,0.00,...,0.00,0.00,0.00,3.43,17.74,4.30,3.79,2.62,0.00,0.0
1,AUBUCCH01,1946,Chet,Aubuchon,DE1,N,30,0.00,2.17,0.00,...,0.00,0.00,0.00,1.53,3.03,0.77,1.17,0.63,0.00,0.0
2,BAKERNO01,1946,Norm,Baker,CH1,N,4,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.0
3,BALTIHE01,1946,Herschel,Baltimore,ST1,N,58,0.00,2.38,0.00,...,0.00,0.00,0.00,1.69,4.53,0.91,1.19,0.55,0.00,0.0
4,BARRJO01,1946,John,Barr,ST1,N,58,0.00,5.09,0.00,...,0.00,0.00,0.00,2.83,7.55,2.14,1.36,0.81,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19107,WILLISC01,2004,Scott,Williams,CLE,N,19,8.00,1.74,0.79,...,0.21,0.32,0.26,1.58,2.16,0.63,0.58,0.47,0.05,0.0
19108,WOODSLO01,2004,Loren,Woods,TOR,N,45,15.82,3.91,1.80,...,0.18,0.87,0.82,1.73,3.64,1.58,1.31,0.76,0.02,0.0
19109,WOODSQY01,2004,Qyntel,Woods,MIA,N,3,13.33,3.33,0.67,...,1.33,0.00,0.67,2.00,4.00,1.67,0.00,0.00,0.00,0.0
19110,WRIGHDO01,2004,Dorell,Wright,MIA,N,3,9.00,2.33,0.00,...,1.33,0.00,1.00,0.67,3.67,1.00,0.33,0.33,1.33,0.0


In [12]:
# For some reason game_date column was not converted
grouped['game_date'] = pd.to_datetime(grouped['game_date'])

In [13]:
def remove_duplicates(lst):
    return list(dict.fromkeys(lst))

grouped['ilkid'] = grouped['ilkid'].apply(remove_duplicates)
grouped = grouped[grouped["game_date"] <= '2005-12-31']
grouped

,game_date,team_id_x,game_id,ilkid
0,1996-11-01,ATL,29600005,"[LAETTCH01, BLAYLMO01, BARRYJO01, NORMAKE01, C..."
1,1996-11-01,BOS,29600001,"[FOXRI01, WESLEDA01, ELLISPE01, WILLIER01, BRI..."
2,1996-11-01,CHI,29600001,"[BROWNRA01, JORDAMI01, RODMADE01, BUECHJU01, S..."
3,1996-11-01,CLE,29600002,"[GEARYRE01, THOMACA01, POTAPVI01, LANGAN02, HI..."
4,1996-11-01,DAL,29600010,"[HARPEDE01, DUMASTO01, GATLICH01, KIDDJA01, MI..."
...,...,...,...,...
20142,2005-12-31,NOK,20500869,"[BROWNPJ01, CLAXTSP01, MASONDE01, WESTDA01, SN..."
20143,2005-12-31,POR,20500435,"[SMITHCH03, PRZYBJO01, RATLITH01, KHRYAVI01, P..."
20144,2005-12-31,SAS,20500431,"[BARRYBR01, MARKSSE01, UDRIHBE01, BOWENBR01, P..."
20145,2005-12-31,SEA,20500437,"[RIDNOLU01, FORTSDA01, POTAPVI01, LEWISRA01, C..."


In [14]:
# Export to be used by another notebook
playerRegularSeasonNormalized.to_csv("player_regular_season_normalized.csv")
grouped.to_csv("team_with_year.csv")